# 앙상블(Ensemble) 학습


In [1]:
from sklearn.datasets import load_breast_cancer
cancer = load_breast_cancer()

In [2]:
from sklearn.preprocessing import MinMaxScaler
cancer_scaledd = MinMaxScaler().fit_transform(cancer.data)

In [3]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    cancer_scaledd, cancer.target, stratify=cancer.target, test_size=0.2, random_state=2021
)

### 1. 앙상블 학습 - Hard Voting

- 로지스틱회귀 
- 서포트 벡터 머신
- k 최근접 이웃

In [4]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier

In [5]:
lrc = LogisticRegression()
svc = SVC() 
knn = KNeighborsClassifier()

In [6]:
from sklearn.ensemble import VotingClassifier

voc = VotingClassifier(
    estimators=[('LR',lrc),('SVC',svc),('KNN',knn)], voting='hard'
)

In [7]:
voc.fit(X_train, y_train)
voc.score(X_test, y_test)

0.9824561403508771

- 개별 분류기의 성능

In [8]:
lrc.fit(X_train, y_train)
svc.fit(X_train, y_train)
knn.fit(X_train, y_train)
lrc.score(X_train, y_train), svc.score(X_train, y_train), knn.score(X_train, y_train)

(0.967032967032967, 0.9824175824175824, 0.9692307692307692)

### 2. 앙상블 학습 - 소프트 복습

In [9]:
# 메소드와 속성값 알아보기

print(dir(lrc))

# __ 는 object에 있는 성질을 뜻하는 것
# like class, method etc...

['C', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_check_n_features', '_estimator_type', '_get_param_names', '_get_tags', '_more_tags', '_predict_proba_lr', '_repr_html_', '_repr_html_inner', '_repr_mimebundle_', '_validate_data', 'class_weight', 'classes_', 'coef_', 'decision_function', 'densify', 'dual', 'fit', 'fit_intercept', 'get_params', 'intercept_', 'intercept_scaling', 'l1_ratio', 'max_iter', 'multi_class', 'n_features_in_', 'n_iter_', 'n_jobs', 'penalty', 'predict', 'predict_log_proba', 'predict_proba', 'random_state', 'score', 'set_params', 'solver', 'sparsify', 'tol', 'verbose', 'warm_start']


In [12]:
#soft voting..?

lrc.predict_proba(X_test[:5])

array([[0.36348222, 0.63651778],
       [0.97162943, 0.02837057],
       [0.1869565 , 0.8130435 ],
       [0.02133462, 0.97866538],
       [0.0548391 , 0.9451609 ]])

In [13]:
# hard voting  ..?

lrc.predict(X_test[:5])

array([1, 0, 1, 1, 1])

In [14]:
svc.predict_proba(X_test[:5])


AttributeError: predict_proba is not available when  probability=False

In [15]:
svc.get_params()

# 학습을 시킬 때 svc를 true로 낮춰야 precit_proba를 사용할 수 있겠다는 것을 알아야함

{'C': 1.0,
 'break_ties': False,
 'cache_size': 200,
 'class_weight': None,
 'coef0': 0.0,
 'decision_function_shape': 'ovr',
 'degree': 3,
 'gamma': 'scale',
 'kernel': 'rbf',
 'max_iter': -1,
 'probability': False,
 'random_state': None,
 'shrinking': True,
 'tol': 0.001,
 'verbose': False}

In [17]:
svc2 = SVC(probability=True)
svc2.fit(X_train, y_train)
svc2.predict_proba(X_test[:5])

# 승률이 올라감

array([[4.80959046e-01, 5.19040954e-01],
       [9.99821632e-01, 1.78368302e-04],
       [1.73769780e-02, 9.82623022e-01],
       [3.77328396e-03, 9.96226716e-01],
       [5.40813546e-03, 9.94591865e-01]])

In [27]:
knn.predict_proba(X_test[:5])

array([[0.4, 0.6],
       [1. , 0. ],
       [0.2, 0.8],
       [0. , 1. ],
       [0. , 1. ]])

In [28]:
voc2 = VotingClassifier(
    estimators=[('LR',lrc),('SVC',svc),('KNN',knn)], voting='soft'
)

In [29]:
voc2.fit(X_train, y_train)
voc2.score(X_test,y_test) 

# 왜 안됨?

AttributeError: predict_proba is not available when  probability=False

In [30]:
voc2.predict(X_test[:5])

AttributeError: predict_proba is not available when  probability=False

- GridSearchCV 적용

In [31]:
params = {
    'LRC__C' : [0.1,1,10],
    'SVC__C' : [0.1,1,10]
}

In [36]:
from sklearn.model_selection import GridSearchCV
grid_voc2 = GridSearchCV(voc2, param_grid='accuracy', cv= 3)
grid_voc2.fit(X_train, y_train)
grid_voc2.best_params_

AttributeError: 'str' object has no attribute 'items'

In [ ]:
params = {
    'LRC__C' : [0.01, 0.05, 0.1, 0.2, 0.5],
    'SVC__C' : [5, 10, 20, 50]
}

grid_voc2 = GridSearchCV(voc2, param_grid='accuracy', cv= 3)
grid_voc2.fit(X_train, y_train)
grid_voc2.best_params_

In [33]:
grid_voc2.best_estimator_.socre(X_test, y_test)

NameError: name 'grid_voc2' is not defined

### 3. Random Forest

- DecisionTree 100개

In [34]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(random_state=2021)
rfc.get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': 2021,
 'verbose': 0,
 'warm_start': False}

In [35]:
rfc.fit(X_train, y_train)
rfc.score(X_test, y_test)

0.9736842105263158